In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import os
import re
import copy
from sklearn.decomposition import PCA
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [2]:
path1 = os.path.join(os.getcwd(), '1/')
path2 = os.path.join(os.getcwd(), '2/')

## Helper Functions

In [96]:
def get_gene_set(mat):
    return set(mat.keys())

In [120]:
def remove_repeated_genes(mat, repeated_genes_dict):
    for g, c in repeated_genes_dict.items():
        gene_df = mat[g]
        mean = pd.DataFrame.sum(gene_df, axis=1)/float(c)
        # drop the repetitions
        rep_idx = np.where(mat.columns.get_loc(g) == True)[0]
        mat.drop(mat.columns[rep_idx], axis=1, inplace=True)
        # insert the averaged col
        mat[g] = copy.deepcopy(mean)
    return mat

In [132]:
def get_intersection_gene_df(mat, intersection_gene_list):
    return mat[intersection_gene_list]

## Dataset 1

In [3]:
mat1 = pd.read_table(os.path.join(path1, 'without_header.txt'))

In [4]:
mat1.head(2)

,SGIP1,NM_032291,0.0,0.0.1,0.0.2,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,...,0.3313,0.3314,0.3315,7.11,0.3316,4.11,0.3317,0.3318,0.3319,0.3320
0,AZIN2,NM_052998+NM_001293562,0.0,0.0,0.0,0.0,0.0,41.519287,0.0,0.0,...,0,0,0,16,0,0,0,0,0,4
1,CLIC4,NM_013943,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,2,20,64,18,0,0,0,0,0,1


In [5]:
mat1.shape

(26270, 7030)

In [6]:
mat1_rpkm_only = mat1.iloc[:, : int(7030/2)]
mat1_rpkm_only.head(2)

,SGIP1,NM_032291,0.0,0.0.1,0.0.2,0.0.3,0.0.4,0.0.5,0.0.6,0.0.7,...,0.0.3312,0.0.3313,0.0.3314,0.0.3315,4.86178584602,0.0.3316,3.10298699593,0.0.3317,0.0.3318,0.0.3319
0,AZIN2,NM_052998+NM_001293562,0.0,0.0,0.0,0.0,0.0,41.519287,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,24.792203,0.0,0.0,0.0,0.0,0.0
1,CLIC4,NM_013943,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,5.185104,14.124313,9.325278,44.743017,13.234809,0.0,0.0,0.0,0.0,0.0


In [7]:
mat1_rpkm_only.shape  

(26270, 3515)

In [8]:
header = pd.read_table(os.path.join(path1, 'header.txt'))

In [9]:
header.head()

,#samples,HP1502401_N13,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8


In [10]:
column_headers = {}
for old, new in zip(mat1_rpkm_only.keys(), header.keys()):
    column_headers[old] = new

In [11]:
mat1_rpkm_only.rename(columns=column_headers, inplace=True)
mat1_rpkm_only.head(2)

,#samples,HP1502401_N13,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
0,AZIN2,NM_052998+NM_001293562,0.0,0.0,0.0,0.0,0.0,41.519287,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,24.792203,0.0,0.0,0.0,0.0,0.0
1,CLIC4,NM_013943,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,5.185104,14.124313,9.325278,44.743017,13.234809,0.0,0.0,0.0,0.0,0.0


In [12]:
#Drop the HP1502401_N13 column
mat1_rpkm_only.drop('HP1502401_N13', axis=1, inplace=True)
mat1_rpkm_only.head(2)

,#samples,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,HP1502401_C14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
0,AZIN2,0.0,0.0,0.0,0.0,0.0,41.519287,0.0,0.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,24.792203,0.0,0.0,0.0,0.0,0.0
1,CLIC4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,5.463685,...,5.185104,14.124313,9.325278,44.743017,13.234809,0.0,0.0,0.0,0.0,0.0


In [13]:
mat1_rpkm_only.tail(2)

,#samples,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,HP1502401_C14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
26268,ERCC_0.01430512:mix1_0.02861023:mix2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.00000,0.0,0.000000,...,0.0,0.000000,0.000000,0.00000,0.00000,0.000000,0.0,0.0,0.0,0.00000
26269,eGFP,157.060811,255.062603,624.804686,0.0,0.0,0.0,6.92024,0.0,2.403371,...,0.0,43.491115,2.871408,4.30535,4.52802,536.509416,0.0,0.0,0.0,1538.08278


In [14]:
# From README, 26180:26271 correspond to data for the 92 external RNA spike-in controls (ERCCs). Deleting these
mat1_rpkm_only.drop(mat1_rpkm_only.tail(93).index,inplace=True)
mat1_rpkm_only.tail()

,#samples,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,HP1502401_C14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
26172,KIR2DL2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26173,KIR2DL4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26174,KIR2DS3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26175,KIR2DS2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
26176,BIVM-ERCC5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# Delete all columns with only 0s
mat1_rpkm_only = mat1_rpkm_only.loc[:, (mat1_rpkm_only != 0).any(axis=0)]
# Delete all rows with only 0s
mat1_rpkm_only = mat1_rpkm_only.loc[(mat1_rpkm_only != 0).any(axis=1), :]

In [16]:
mat1_rpkm_only.shape

(26177, 3514)

In [97]:
mat1_rpkm_only.set_index('#samples',inplace=True)

In [99]:
mat1_rpkm_only.head(2)

,HP1502401_D14,HP1502401_F14,HP1502401_J13,HP1502401_B13,HP1502401_H13,HP1502401_J14,HP1502401_B14,HP1502401_A14,HP1502401_C14,HP1502401_G14,...,HP1525301T2D_O10,HP1526901T2D_H2,HP1526901T2D_I16,HP1526901T2D_F7,HP1526901T2D_I23,HP1525301T2D_K3,HP1525301T2D_J10,HP1526901T2D_N8,HP1526901T2D_O11,HP1526901T2D_A8
#samples,,,,,,,,,,,,,,,,,,,,,
AZIN2,0.0,0.0,0.0,0.0,0.0,41.519287,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,24.792203,0.0,0.0,0.0,0.0,0.0
CLIC4,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,5.463685,0.230823,...,5.185104,14.124313,9.325278,44.743017,13.234809,0.0,0.0,0.0,0.0,0.0


In [121]:
mat1T = mat1_rpkm_only.T
mat1T.head(2)

#samples,AZIN2,CLIC4,AGBL4,NECAP2,SLC45A1,TGFBR3,DBT,RFWD2,C1orf21,PRUNE,...,KIR2DL5A,KIR3DS1,KIR2DL5B,KIR2DS2,KIR2DS1,KIR2DL2,KIR2DL4,KIR2DS3,KIR2DS2,BIVM-ERCC5
HP1502401_D14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
HP1502401_F14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [122]:
# compare number of unique genes with total cols
genes1 = get_gene_set(mat1T)
print('# of unique genes = %s' %(len(genes1)))
print('# of cols = %s' %(len(mat1T.keys())))

# of unique genes = 25452
# of cols = 26177


In [123]:
# Merging same gene cols by taking mean 
gene_occ_dict = {}
for g in mat1T.keys():
    if g not in gene_occ_dict.keys():
        gene_occ_dict[g] = 1
    else:
        gene_occ_dict[g] += 1

In [124]:
repeated_genes_dict = {}
for g, c in gene_occ_dict.items():
    if c > 1:
        repeated_genes_dict[g] = c

In [125]:
mat1T = remove_repeated_genes(mat1T, repeated_genes_dict)
mat1T.shape

(3513, 25452)

## Dataset 2

In [79]:
mat2 = pd.read_table(os.path.join(path2, 'GSE81608_human_islets_rpkm.txt'))

In [80]:
mat2.head(3)

,gene.id,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,Sample_9,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
0,1,47.3396,24.0458,2.2743,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.3304,2.4857,0.0,8.1498,0.0,0.5372,31.1225,0.0000,0.5788
1,2,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.254,0.0,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,47.4664,0.0000
2,3,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000


In [81]:
mat2['gene.id'] = mat2['gene.id'].astype(int)

### Mapping from entrez id (gene.id) to symbols

In [21]:
temp = mat2['gene.id'].astype(str)
with open(os.path.join(path2, 'entrez.txt'), 'w') as f:
    for i in temp.values:
        f.write(str(i) + "\n")

In [82]:
e2s = {}
with open(os.path.join(path2, 'symbols.txt') ,'rb') as f: # obtained by running an R script
    for entrez, sym in zip(temp, f.readlines()):
        e2s[int(entrez)] = str(sym)[2:-3]

In [83]:
mat2.insert(loc=1, column='gene_symbol', value=str)
mat2.head(2)

,gene.id,gene_symbol,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
0,1,<class 'str'>,47.3396,24.0458,2.2743,0.0,0.0,0.0,0.0,0.000,...,0.0,0.3304,2.4857,0.0,8.1498,0.0,0.5372,31.1225,0.0000,0.5788
1,2,<class 'str'>,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.254,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,47.4664,0.0000


In [84]:
for i in mat2.index:
    gene_id = mat2.iloc[i,0]
    mat2.iloc[i,1] = e2s[gene_id]

In [85]:
mat2.head(2)

,gene.id,gene_symbol,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
0,1,A1BG,47.3396,24.0458,2.2743,0.0,0.0,0.0,0.0,0.000,...,0.0,0.3304,2.4857,0.0,8.1498,0.0,0.5372,31.1225,0.0000,0.5788
1,2,A2M,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.254,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,47.4664,0.0000


In [86]:
mat2.drop('gene.id', axis=1, inplace=True)
mat2.head(2)

,gene_symbol,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,Sample_9,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
0,A1BG,47.3396,24.0458,2.2743,0.0,0.0,0.0,0.0,0.000,0.0,...,0.0,0.3304,2.4857,0.0,8.1498,0.0,0.5372,31.1225,0.0000,0.5788
1,A2M,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.254,0.0,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,47.4664,0.0000


In [87]:
mat2.set_index('gene_symbol', inplace=True)

In [88]:
mat2.head(2)

,Sample_1,Sample_2,Sample_3,Sample_4,Sample_5,Sample_6,Sample_7,Sample_8,Sample_9,Sample_10,...,Sample_1591,Sample_1592,Sample_1593,Sample_1594,Sample_1595,Sample_1596,Sample_1597,Sample_1598,Sample_1599,Sample_1600
gene_symbol,,,,,,,,,,,,,,,,,,,,,
A1BG,47.3396,24.0458,2.2743,0.0,0.0,0.0,0.0,0.000,0.0,2.0351,...,0.0,0.3304,2.4857,0.0,8.1498,0.0,0.5372,31.1225,0.0000,0.5788
A2M,0.0000,0.0000,0.0000,0.0,0.0,0.0,0.0,0.254,0.0,0.0000,...,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0000,0.0000,47.4664,0.0000


In [89]:
mat2T = mat2.T
mat2T.head(2)

gene_symbol,A1BG,A2M,A2MP1,NAT1,NAT2,NATP,SERPINA3,AADAC,AAMP,AANAT,...,NA,LOC101929767,OSMR-AS1,LINC01063,LOC101929770,LINC01655,LOC101930100,NA,NA,LOC102724238
Sample_1,47.3396,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,31.2047,0.0,...,0.0,5.1983,0.0,0.0,0.0,0.0,0.0,0.2423,0.2423,0.0
Sample_2,24.0458,0.0,0.0,0.0,0.0,0.0,14.0425,0.0,0.0000,0.0,...,0.0,0.1064,0.0,0.0,0.0,0.0,0.0,0.0000,0.0000,0.0


In [92]:
# Dropping NA gene columns
drop_idx = np.where(mat2T.columns.values =='NA')[0]

In [93]:
len(drop_idx)

1601

In [94]:
mat2T.drop(mat2T.columns[drop_idx], axis=1, inplace=True)
mat2T.head(2)

gene_symbol,A1BG,A2M,A2MP1,NAT1,NAT2,NATP,SERPINA3,AADAC,AAMP,AANAT,...,PIK3IP1-AS1,LOC101929762,LINC01585,LOC101929767,OSMR-AS1,LINC01063,LOC101929770,LINC01655,LOC101930100,LOC102724238
Sample_1,47.3396,0.0,0.0,0.0,0.0,0.0,0.0000,0.0,31.2047,0.0,...,0.0,0.1284,0.0,5.1983,0.0,0.0,0.0,0.0,0.0,0.0
Sample_2,24.0458,0.0,0.0,0.0,0.0,0.0,14.0425,0.0,0.0000,0.0,...,0.0,0.1236,0.0,0.1064,0.0,0.0,0.0,0.0,0.0,0.0


In [95]:
mat2T.shape

(1600, 38250)

In [127]:
# Delete all columns with only 0s
mat2T = mat2T.loc[:, (mat2T != 0).any(axis=0)]
# Delete all rows with only 0s
mat2T = mat2T.loc[(mat2T != 0).any(axis=1), :]
mat2T.shape

(1600, 32453)

No cells were removed. 38250 - 32453 = 5797 genes were removed

In [128]:
# Checking for repetitive cols
genes2 = get_gene_set(mat2T)
len(genes2)

32453

Clearly no repetitive cols in dataset 2

## Intersection Genes

In [129]:
intersection_genes = genes1.intersection(genes2)

In [130]:
len(intersection_genes)

20964

In [133]:
mat1_intrscn = get_intersection_gene_df(mat1T, list(intersection_genes))
mat2_intrscn = get_intersection_gene_df(mat2T, list(intersection_genes))

In [134]:
print(mat1_intrscn.shape)
print(mat2_intrscn.shape)

(3513, 20964)
(1600, 20964)


In [135]:
mat1_intrscn.to_pickle(os.path.join(path1,'1_mat1.pkl'))
mat2_intrscn.to_pickle(os.path.join(path2,'1_mat2.pkl'))